# First attempt to create a new representation method for a candidates

In [1]:
# %load_ext lab_black

### Imports

In [2]:
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_packed_sequence
import pandas as pd

from model.encoder import CandidateEncoder
from model.embedder import EmbedderType
from dataset.utils import pad_collate
from dataset.dataset import SellersDataset
from dataset.language import Lang

### Constants

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EMBEDDER_NAME = EmbedderType.LANG

In [4]:
dataset = SellersDataset(
    embedder_name=EMBEDDER_NAME, data_path="data/extracted_sellers.json", device=DEVICE
)
dataset.prepare_dataset()

[2022-05-16 20:56:06,008] {dataset.py:133} INFO - Preparing dataset
[2022-05-16 20:56:06,008] {dataset.py:160} INFO - Detecting languages:


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 15.49it/s]

[2022-05-16 20:56:06,269] {dataset.py:165} INFO - Detected languages:
[2022-05-16 20:56:06,270] {dataset.py:166} INFO - lang
en    4
Name: lang, dtype: int64
[2022-05-16 20:56:06,271] {dataset.py:167} INFO - Removing rows not written in english
[2022-05-16 20:56:06,272] {dataset.py:171} INFO - Removed 0 rows



100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 8494.79it/s]

[2022-05-16 20:56:06,283] {dataset.py:347} INFO - Adding language for languages_str


...


100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 19463.13it/s]

[2022-05-16 20:56:06,287] {dataset.py:347} INFO - Adding language for skills_str



100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 14794.72it/s]

[2022-05-16 20:56:06,289] {dataset.py:347} INFO - Adding language for description_str



100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 7814.26it/s]


## Prepare data

### Prepare dataloader

In [5]:
loader = DataLoader(dataset, batch_size=3, collate_fn=pad_collate)

In [6]:
encoder = CandidateEncoder(
    latent_dim=64,
    lstm_hidden_dim=128,
    num_layers_lstm=1,
    hidden_dims=[64],
    embedding_size=dataset.embedder.size,
    num_words=dataset.lang.n_words,
    embedder_name=EMBEDDER_NAME,
    device=DEVICE,
).to(DEVICE)

In [7]:
out = encoder(next(iter(loader)).cuda())

/media/patryk/TOSHIBA EXT/Studia/SztucznaInteligencja/PracaMagisterska/master-thesis/seq2seq/dataset/dataset.py:329: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["skills_str"] = (


In [8]:
out[0].shape, out[1].shape, out[2].shape

(torch.Size([3, 64]), torch.Size([3, 64]), torch.Size([3, 133, 256]))